In [1]:
import os
import tempfile
from langchain import PromptTemplate, HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
file_path = f'C:/Users/user/Desktop/AES automatic electronic Slide user guide.pdf'

In [3]:
import pypdf
Read_pdf = open(file_path, 'rb')
pdf = pypdf.PdfReader(Read_pdf)
len(pdf.pages)

7

In [4]:
page_number = 5
page = pdf.pages[page_number]
content = page.extract_text()
print(len(content))

1508


In [5]:
loader = PyPDFLoader(file_path)
pdf = loader.load()

In [6]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
doc_splitter = RecursiveCharacterTextSplitter(chunk_size = 450, chunk_overlap = 50)
pdf_chunks = doc_splitter.split_documents(pdf)
len(pdf_chunks)

38

In [7]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_xxx"
pinecone.init(api_key="xxx", environment="northamerica-northeast1-gcp")

In [13]:
# Create embeddings for the pages and insert into pinecone database
model_name = "bert-base-uncased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


































































































































































































No sentence-transformers model found with name C:\Users\user/.cache\torch\sentence_transformers\bert-base-uncased. Creating a new one with MEAN pooling.


In [18]:

index_name = "marketing-pr"
pinecone.create_index(index_name, dimension=768, metric="cosine")
index = pinecone.Index(index_name=index_name)

In [19]:
from langchain.vectorstores import Pinecone
# Create document search database
db =  Pinecone.from_documents(pdf_chunks, embeddings, index_name=index_name)

In [119]:
retriever = db.as_retriever()
# Define the template for the language model prompt
template =  """
            You are a virtual technician designed to provide support and information to clients on automatic sliding door systems.\
            Your objective is to provide accurate and simplified responses to a wide range of \
            automatic sliding door systems questions, based on your knowledge base. \

            If a user's query indicates an error code or number, go through document to give possible reasons and remedies to the issue\
            

            In your responses, ensure a tone of professionalism, understanding and expertise. Provide users with \
            possible solutions and stop gaps to their issues pending when the physical technician will arrive \
        

            Here are some specific interaction scenarios to guide your responses:
            - If the- user asks what you can do, respond with "I'm a virtual technician here to provide \
            first aid support and information on the technical issue you are experiencing. How can I assist you?"
            - If the user starts with a greeting, respond with 'Hello! How are you doing today? How can I assist you?' \
            or something related to that
            - If a user shares their name, use it in your responses when appropriate, to cultivate a more personal and \
            comforting conversation.

            - If the exact question is not available, ask the user to reboot the system and then provide a suggestion based on the second nearest answer from the pdf.\
            - If a user asks a question that is unrelated to automated systems, respond with \
            'This question is out of my scope as I'm built mainly as a first aid support for our electronic systems in your facility \
            Could you please ask a question related to our automated systems?'

            {context}
            Question: {question}
            Answer:"""

# Create a prompt template
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [125]:
llm=HuggingFaceHub(repo_id='meta-llama/Llama-2-7b-chat-hf', model_kwargs={"temperature":1e-10})
# falcon7b and flan large

In [126]:
chain_type_kwargs = {"prompt": prompt}
#Initialize the OpenAI module, load and run the retrieval QA chain
chain = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
#chain_type_kwargs=chain_type_kwargs,
verbose=True
)

In [48]:
user_question = input("How can we assist you today?")
user_question

'I have an error E7 16'

In [128]:
answer = chain.run(user_question)
answer



> Entering new  chain...


ValueError: Error raised by inference API: Model requires a Pro subscription